<a href="https://colab.research.google.com/github/Tanush005/100-days-of-deep-learning/blob/main/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow numpy pandas scikit-learn matplotlib



In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pip install tensorflow numpy pandas scikit-learn imbalanced-learn xgboost matplotlib seaborn

In [ ]:
# Load dataset
data = pd.read_csv('synthetic_eth_transactions.csv')

# Display dataset info
print(data.info())
print(data.head())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis

# Load transaction data (Assuming a CSV file)
df = pd.read_csv("/content/synthetic_ethereum_transactions (1).csv")

# Display basic information
print("Dataset Overview:\n")
print(df.info())  # Check data types & missing values
print("\nSummary Statistics:\n", df.describe())

# Checking for missing values
missing_values = df.isnull().sum()
print("\nMissing Values:\n", missing_values[missing_values > 0])

# Checking for duplicate transactions
duplicates = df.duplicated().sum()
print(f"\nNumber of Duplicate Records: {duplicates}")


# Convert 'timestamp' to datetime (if applicable)
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(by='timestamp')  # Sort transactions chronologically

# Convert categorical features (if needed)
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].astype('category')

# Handling outliers using IQR method for transaction amounts
if 'amount' in df.columns:
    Q1 = df['amount'].quantile(0.25)
    Q3 = df['amount'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    df = df[(df['amount'] >= lower_bound) & (df['amount'] <= upper_bound)]  # Remove outliers



# 1. Transaction Amount Distribution
plt.figure(figsize=(10, 5))
sns.histplot(df['Amount'], bins=50, kde=True, color='blue')
plt.title('Transaction Amount Distribution')
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency')
plt.show()

# 2. Frequency of Transactions Per Day
if 'timestamp' in df.columns:
    df['date'] = df['timestamp'].dt.date
    daily_txn_counts = df.groupby('date').size()

    plt.figure(figsize=(12, 5))
    sns.lineplot(x=daily_txn_counts.index, y=daily_txn_counts.values, marker="o")
    plt.title('Daily Transaction Frequency')
    plt.xlabel('Date')
    plt.ylabel('Number of Transactions')
    plt.xticks(rotation=45)
    plt.show()

# 3. Time Intervals Between Transactions
if 'timestamp' in df.columns:
    df['time_diff'] = df['timestamp'].diff().dt.total_seconds()

    plt.figure(figsize=(10, 5))
    sns.histplot(df['time_diff'].dropna(), bins=50, kde=True, color='green')
    plt.title('Distribution of Time Intervals Between Transactions')
    plt.xlabel('Time Interval (seconds)')
    plt.ylabel('Frequency')
    plt.show()



In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
df = pd.read_csv('/content/synthetic_ethereum_transactions (1).csv')

# Function to shorten addresses
def shorten_address(addr):
    return addr[:4] + "..." + addr[-4:] if len(addr) > 10 else addr

# Count the number of transactions per sender and receiver
top_senders = df['Sender'].value_counts().head(10).reset_index()
top_senders.columns = ['Sender', 'Transactions']

top_receivers = df['Receiver'].value_counts().head(10).reset_index()
top_receivers.columns = ['Receiver', 'Transactions']

# Apply function to sender & receiver labels
top_senders["Short_Sender"] = top_senders["Sender"].apply(shorten_address)
top_receivers["Short_Receiver"] = top_receivers["Receiver"].apply(shorten_address)

# Create figure
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart for top senders
ax[0].pie(top_senders['Transactions'], labels=top_senders['Short_Sender'], autopct='%1.1f%%',
          startangle=140, colors=plt.cm.coolwarm(range(10)), textprops={'fontsize': 9})
ax[0].set_title("Top 10 Senders (Pie Chart)")

# Pie chart for top receivers
ax[1].pie(top_receivers['Transactions'], labels=top_receivers['Short_Receiver'], autopct='%1.1f%%',
          startangle=140, colors=plt.cm.viridis(range(10)), textprops={'fontsize': 9})
ax[1].set_title("Top 10 Receivers (Pie Chart)")

# Show plot
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load dataset
df = pd.read_csv("synthetic_eth_transactions.csv")

# Ensure timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ------------------------------
# Feature Engineering
# ------------------------------

# 1. **Transaction Amount Features**
df['log_amount'] = np.log1p(df['amount'])  # Log-transform to reduce skewness
df['large_txn_flag'] = (df['amount'] > df['amount'].quantile(0.99)).astype(int)  # Flag top 1% transactions

# 2. **Frequency-Based Features**
df['daily_txn_count'] = df.groupby(['sender', df['timestamp'].dt.date])['transaction_id'].transform('count')
df['hourly_txn_count'] = df.groupby(['sender', df['timestamp'].dt.hour])['transaction_id'].transform('count')

# # 3. **Time-Based Features**
# df['is_night_txn'] = df['hour_of_day'].apply(lambda x: 1 if x < 6 or x > 22 else 0)  # Transactions between 10 PM - 6 AM
# df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)  # Weekend transactions

# 4. **Transaction Speed Features**
df['txn_speed'] = df.groupby('sender')['timestamp'].diff().dt.total_seconds()  # Time between transactions
df['avg_txn_speed'] = df.groupby('sender')['txn_speed'].transform('mean')  # Average transaction speed per sender

# # 5. **Gas Usage & Transaction Fee Features**
# df['effective_gas_price'] = df['gas_price'] * df['gas_used']
# df['relative_gas_usage'] = df['gas_used'] / df['gas_used'].max()
# df['txn_fee_to_amount_ratio'] = df['transaction_fee'] / (df['amount'] + 1)  # Normalized fee-to-amount ratio

# 6. **Behavioral Features (Deviation from Historical Patterns)**
df['sender_avg_txn_amt'] = df.groupby('sender')['amount'].transform('mean')
df['sender_txn_amt_std'] = df.groupby('sender')['amount'].transform('std')
df['txn_amt_deviation'] = (df['amount'] - df['sender_avg_txn_amt']) / (df['sender_txn_amt_std'] + 1e-9)  # Z-score

# 7. **Network Features**
df['sender_txn_count'] = df.groupby('sender')['transaction_id'].transform('count')  # How many transactions has the sender made?
df['receiver_txn_count'] = df.groupby('receiver')['transaction_id'].transform('count')  # Transactions received
df['unique_receivers'] = df.groupby('sender')['receiver'].transform('nunique')  # How many unique addresses a sender has sent money to
df['unique_senders'] = df.groupby('receiver')['sender'].transform('nunique')  # Unique senders

# 8. **Anomaly Detection Flags**
df['high_freq_flag'] = (df['daily_txn_count'] > df['daily_txn_count'].quantile(0.99)).astype(int)  # Unusually frequent transactions
# df['high_txn_fee_flag'] = (df['txn_fee_to_amount_ratio'] > df['txn_fee_to_amount_ratio'].quantile(0.99)).astype(int)  # High fee transactions
df['behavioral_anomaly_flag'] = (df['txn_amt_deviation'].abs() > 3).astype(int)  # Transactions outside 3-sigma range

# Drop unnecessary intermediate columns if needed
df.drop(columns=['txn_speed'], inplace=True)

# Save the engineered dataset
df.to_csv("transaction_data_featured.csv", index=False)

print("Feature Engineering Complete. Saved as 'transaction_data_featured.csv'.")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the engineered dataset
df = pd.read_csv("/content/transaction_data_featured.csv")

# Set up the style for plots
sns.set(style="whitegrid")
plt.figure(figsize=(12, 8))


# Log-transformed amount distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['log_amount'], bins=50, kde=True, color='blue')
plt.title('Distribution of Log-Transformed Transaction Amounts')
plt.xlabel('Log(Amount)')
plt.ylabel('Frequency')
plt.show()

# Large transaction flag distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='large_txn_flag', data=df, palette='Set2')
plt.title('Distribution of Large Transaction Flags')
plt.xlabel('Large Transaction Flag (1 = Large)')
plt.ylabel('Count')
plt.show()


# Daily transaction count distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['daily_txn_count'], bins=50, kde=True, color='green')
plt.title('Distribution of Daily Transaction Counts')
plt.xlabel('Daily Transaction Count')
plt.ylabel('Frequency')
plt.show()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Distribution of Transaction Amount (Before & After Log Transformation)
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(df['amount'], bins=50, ax=ax[0], kde=True)
ax[0].set_title("Original Transaction Amounts")

sns.histplot(df['log_amount'], bins=50, ax=ax[1], kde=True)
ax[1].set_title("Log Transformed Transaction Amounts")

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only numeric columns
numeric_df = df.select_dtypes(include=['number'])

# Compute correlation matrix
corr_matrix = numeric_df.corr()

# Plot heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Matrix")
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA
from keras.models import Model, Sequential
from keras.layers import Input, Dense

# Load data
df = pd.read_csv("synthetic_eth_transactions.csv")

# Select numeric features for clustering & anomaly detection
numeric_features = df.select_dtypes(include=['number'])

# Standardizing the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_features)

## K-Means Clustering
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df['kmeans_cluster'] = kmeans.fit_predict(scaled_data)

## DBSCAN Clustering (Detects small outlier clusters)
dbscan = DBSCAN(eps=1.5, min_samples=5)
df['dbscan_cluster'] = dbscan.fit_predict(scaled_data)


## Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)
df['iso_forest_score'] = iso_forest.fit_predict(scaled_data)

## One-Class SVM
oc_svm = OneClassSVM(nu=0.01, kernel="rbf", gamma='auto')
df['oc_svm_score'] = oc_svm.fit_predict(scaled_data)


input_dim = scaled_data.shape[1]


autoencoder = Sequential([
    Dense(32, activation='relu', input_shape=(input_dim,), kernel_initializer='he_normal'),
    Dense(16, activation='relu', kernel_initializer='he_normal'),
    Dense(8, activation='relu', kernel_initializer='he_normal'),
    Dense(16, activation='relu', kernel_initializer='he_normal'),
    Dense(32, activation='relu', kernel_initializer='he_normal'),
    Dense(input_dim, activation='linear')  # Reconstruction
])

autoencoder.compile(optimizer='adam', loss='mse')

from keras.callbacks import Callback

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

history = autoencoder.fit(scaled_data, scaled_data, epochs=10, batch_size=32, shuffle=True, verbose=0, callbacks=[LossHistory(), early_stopping])

history = autoencoder.fit(scaled_data, scaled_data,epochs=10, batch_size=32, shuffle=True, verbose=0,callbacks=[LossHistory(), early_stopping])


# Compute reconstruction error
reconstructed = autoencoder.predict(scaled_data)
reconstruction_error = np.mean(np.abs(reconstructed - scaled_data), axis=1)
df['autoencoder_score'] = reconstruction_error

G = nx.DiGraph()

for _, row in df.iterrows():
    G.add_edge(row['sender'], row['receiver'], weight=row['amount'])

# Compute Node Features (PageRank & Betweenness Centrality)

import networkx as nx

# Check if the graph is weakly connected (ignoring edge directions)
if not nx.is_weakly_connected(G):
    print("Warning: Graph is disconnected! Computing centrality on the largest weakly connected component.")

    # Get the largest weakly connected component and convert it back to a DiGraph
    largest_component = max(nx.weakly_connected_components(G), key=len)
    G = G.subgraph(largest_component).copy()

# Compute Betweenness Centrality (Optimized for Large Graphs)
import networkx as nx
from joblib import Parallel, delayed
import numpy as np

def compute_betweenness(G, node_subset):
    """Compute betweenness centrality for a subset of nodes."""
    return nx.betweenness_centrality(G, k=None, normalized=True, endpoints=True)

# Ensure the graph is connected
if not nx.is_connected(G.to_undirected()):
    print("Warning: Graph is disconnected! Computing on the largest component.")
    G = G.subgraph(max(nx.connected_components(G.to_undirected()), key=len))

# Split nodes for parallel computation
nodes = list(G.nodes())
num_chunks = 4  # Adjust based on your system's CPU
chunks = np.array_split(nodes, num_chunks)

# Run in parallel
betweenness_results = Parallel(n_jobs=num_chunks)(
    delayed(compute_betweenness)(G, chunk) for chunk in chunks
)

# Merge results
betweenness_centrality = {node: 0 for node in nodes}
for result in betweenness_results:
    for node, value in result.items():
        betweenness_centrality[node] += value

print(" Parallel Betweenness Centrality Computed Successfully!")


betweenness = {node: sum(res.get(node, 0) for res in betweenness_results) for node in G.nodes()}


print("Betweenness centrality computed successfully!")




pagerank = nx.pagerank(G, alpha=0.85)

df['pagerank'] = df['sender'].map(pagerank)
df['betweenness'] = df['sender'].map(betweenness)



# Handle missing values for unseen senders
df.fillna({'pagerank': 0, 'betweenness': 0}, inplace=True)

# ---------- 5. Hybrid Anomaly Score ----------
df['anomaly_score'] = (
    (df['iso_forest_score'] == -1).astype(int) +
    (df['oc_svm_score'] == -1).astype(int) +
    (df['autoencoder_score'] > df['autoencoder_score'].quantile(0.99)).astype(int) +
    (df['pagerank'] > df['pagerank'].quantile(0.99)).astype(int) +
    (df['betweenness'] > df['betweenness'].quantile(0.99)).astype(int) +
    (df['dbscan_cluster'] == -1).astype(int)
)

df['fraud_flag'] = (df['anomaly_score'] > 2).astype(int)  # Threshold for fraud detection

# ---------- 6. Visualize Results ----------
sns.histplot(df['anomaly_score'], bins=20, kde=True)
plt.title("Anomaly Score Distribution")
plt.show()

# Output top 10 suspected fraudulent transactions
fraudulent_transactions = df[df['fraud_flag'] == 1]
print(fraudulent_transactions[['transaction_id', 'sender', 'receiver', 'amount', 'fraud_flag']].head(10))


In [ ]:
!pip install torch torchvision torchaudio \
            torch-geometric \
            networkx numpy pandas gym stable-baselines3 \
            scikit-learn matplotlib

!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv \
            -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install "shimmy>=2.0"


import os
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
import numpy as np
import pandas as pd
import gym
import torch.nn.functional as F
from gym import spaces
from sklearn.preprocessing import MinMaxScaler
from stable_baselines3 import PPO
import seaborn as sns
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.decomposition import PCA
# Load dataset
file_path = "ss.csv"
if not os.path.exists(file_path):
    print(" Uploading dataset. Please select your file.")
    from google.colab import files
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]

df = pd.read_csv(file_path)

# Check necessary columns
expected_cols = ['Sender', 'Receiver', 'Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']
missing_cols = [col for col in expected_cols if col not in df.columns]
if missing_cols:
    raise KeyError(f"Missing columns in dataset: {missing_cols}")

# Normalize transaction features
scaler = MinMaxScaler()
df[['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']] = scaler.fit_transform(
    df[['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']]
)

# Create node index mapping
unique_addresses = set(df['Sender'].astype(str).tolist() + df['Receiver'].astype(str).tolist())
address_map = {addr: i for i, addr in enumerate(unique_addresses)}

df['sender_idx'] = df['Sender'].astype(str).map(address_map)
df['receiver_idx'] = df['Receiver'].astype(str).map(address_map)

# Prepare LSTM Data (Sequential Patterns)
def prepare_lstm_data(df, seq_length=10):
    sequences = []
    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i+seq_length][['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']].values
        sequences.append(seq)
    return torch.tensor(sequences, dtype=torch.float32)

lstm_data = prepare_lstm_data(df)

# Define LSTM Model
class LSTMFraudDetector(nn.Module):
    def _init_(self, input_dim, hidden_dim, output_dim):
        super(LSTMFraudDetector, self)._init_()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return torch.tanh(self.fc(hn[-1]))  # Output anomaly score

lstm_model = LSTMFraudDetector(input_dim=4, hidden_dim=64, output_dim=1)

# Prepare Graph Data
graph = nx.DiGraph()
for _, row in df.iterrows():
    graph.add_edge(row['sender_idx'], row['receiver_idx'], weight=row['Amount'])

graph_data = Data(
    edge_index=torch.tensor(list(graph.edges)).t().contiguous(),
    edge_attr=torch.tensor([graph[u][v]['weight'] for u, v in graph.edges], dtype=torch.float32)
)


class GNNFraudDetector(nn.Module):
    def _init_(self, input_dim, hidden_dim, output_dim):
        super(GNNFraudDetector, self)._init_()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, node_features, edge_index):
        """ Forward pass with correct node features. """
        x = self.conv1(node_features, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

    def get_node_embeddings(self, node_features, edge_index):
       with torch.no_grad():
          x = self.conv1(node_features, edge_index)
          x = F.relu(x)
          embeddings = self.conv2(x, edge_index)
       return embeddings

gnn_model = GNNFraudDetector(input_dim=4, hidden_dim=64, output_dim=64)

# Define RL Environment
class FraudDetectionEnv(gym.Env):
    def _init_(self, df, gnn_model, lstm_model):
        super(FraudDetectionEnv, self)._init_()
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=-1, high=1, shape=(65,), dtype=np.float32)

        self.df = df
        self.gnn_model = gnn_model
        self.lstm_model = lstm_model
        self.current_index = 0

    def reset(self):
      self.current_index = 0
      return self._get_features(self.df.iloc[self.current_index])

    def _get_features(self, transaction):
      if isinstance(transaction, pd.DataFrame):
        transaction = transaction.iloc[0]  # Convert DataFrame to Series

      sender_idx = int(transaction['sender_idx'])
      receiver_idx = int(transaction['receiver_idx'])


      edge_index = torch.tensor([[sender_idx], [receiver_idx]],
                                dtype=torch.long)

      edge_attr = torch.tensor([float(transaction['Amount'])], dtype=torch.float32)

      num_nodes = max(sender_idx, receiver_idx) + 1
      node_features = torch.randn(num_nodes, 4)

      gnn_embeddings = self.gnn_model(node_features,edge_index )
      if sender_idx >= gnn_embeddings.shape[0]:
        sender_idx = 0

      gnn_embedding1 = gnn_embeddings[sender_idx].detach().cpu().numpy()
      gnn_embedding = gnn_embedding1.flatten()
      print(f"GNN embedding shape: {gnn_embedding.shape}")


      lstm_input_data = transaction[['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']].astype(float).fillna(0).values

      lstm_input = torch.tensor(lstm_input_data.reshape(1, 1, -1),
                                dtype=torch.float32)
      lstm_score = self.lstm_model(lstm_input).detach().cpu().numpy().flatten()
      print(f"LSTM score shape: {lstm_score.shape}")

      feature_vector = np.concatenate((gnn_embedding, lstm_score))

      print(f"Final feature vector shape: {feature_vector.shape}")

      if feature_vector.shape != (65,):
        raise ValueError(f"Feature vector shape mismatch: Expected (65,), got{feature_vector.shape}")

      return feature_vector

    def step(self, action):
        transaction = self.df.iloc[self.current_index]


        lstm_input = torch.tensor([[transaction[['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']].values]], dtype=torch.float32)
        anomaly_score = self.lstm_model(lstm_input).detach().cpu().numpy().item()

        reward = 0
        if anomaly_score > 0.5:  # High anomaly score → likely fraud
            if action == 1: reward = 10  # Flagging an anomaly is good
            elif action == 2: reward = 15  # Blocking an anomaly is best
            else: reward = -10  # Ignoring an anomaly is bad
        else:
            if action == 1: reward = -5  # False positive penalty
            elif action == 2: reward = -10  # Blocking legit transactions is bad
            else: reward = 5  # Ignoring normal transaction is good

        self.current_index += 1
        done = self.current_index >= len(self.df) - 1
        next_state = self._get_features(self.df.iloc[self.current_index]) if not done else np.zeros(65)
        return next_state, reward, done, {}





def plot_lstm_fraud_scores(df, lstm_model):
    fraud_scores = []

    for i in range(len(df)):
        lstm_input_data = df.iloc[i][['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']].astype(float).values
        lstm_input = torch.tensor(lstm_input_data.reshape(1, 1, -1), dtype=torch.float32)
        score = lstm_model(lstm_input).detach().cpu().numpy().flatten()[0]
        fraud_scores.append(score)

    # ✅ Ensure fraud_scores has the same length as df before adding to DataFrame
    df['Fraud_Score'] = fraud_scores
    # Plot using seaborn
    plt.figure(figsize=(10, 5))
    sns.histplot(df['Fraud_Score'], bins=50, kde=True)
    plt.xlabel("LSTM Fraud Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of Fraud Scores from LSTM")
    plt.show()

def plot_gnn_embeddings(df, lstm_model, gnn_model):
    fraud_scores = []
    for i in range(len(df)):
      lstm_input_data = df.iloc[i][['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']].astype(float).values
      lstm_input = torch.tensor(lstm_input_data.reshape(1, 1, -1), dtype=torch.float32)
      score = lstm_model(lstm_input).detach().cpu().numpy().flatten()[0]
      fraud_scores.append(score)

    node_features = torch.tensor(df[['Amount', 'Gas_Price', 'Gas_Used', 'Sender_Receiver_Count']].astype(float).values,
      dtype=torch.float32)
    unique_addresses = pd.unique(df[['Sender', 'Receiver']].values.ravel())
    address_to_idx = {addr: idx for idx, addr in enumerate(unique_addresses)}

    df['Sender_idx'] = df['Sender'].map(address_to_idx)
    df['Receiver_idx'] = df['Receiver'].map(address_to_idx)

    df[['Sender_idx', 'Receiver_idx']] = df[['Sender_idx', 'Receiver_idx']].fillna(-1).astype(int)

    edge_index = torch.tensor(df[['Sender_idx', 'Receiver_idx'
                            ]].values.T, dtype=torch.long)

    node_features = torch.tensor(df[['Amount', 'Gas_Price', 'Gas_Used','Sender_Receiver_Count']].values, dtype=torch.float32)

    embeddings = gnn_model.get_node_embeddings(node_features, edge_index)
    embeddings = embeddings.detach().cpu().numpy()
    df['Fraud_Score'] = fraud_scores
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=embeddings[:, 0], y=embeddings[:, 1], hue=df['Fraud_Score'], palette="coolwarm", alpha=0.7)
    plt.xlabel("GNN Embedding Dimension 1")
    plt.ylabel("GNN Embedding Dimension 2")
    plt.title("GNN Node Embeddings Visualization")
    plt.show()


plot_lstm_fraud_scores(df, lstm_model)
plot_gnn_embeddings(df,lstm_model, gnn_model)

# After running RL agent
actions = []
env_instance = FraudDetectionEnv(df, gnn_model, lstm_model)
vec_env = DummyVecEnv([lambda: env_instance])
ppo_model = PPO("MlpPolicy", vec_env, verbose=1)
ppo_model.learn(total_timesteps=10000)

for i in range(len(df)):
    features = env_instance._get_features(df.iloc[i])
    action, _ = ppo_model.predict(features)
    actions.append(action)

actions = np.array(actions)

def plot_rl_agent_decisions(actions):
    plt.figure(figsize=(12, 5))
    sns.histplot(actions, bins=3, discrete=True, kde=False, color="blue")
    plt.xticks([0, 1, 2], ["Ignore", "Flag", "Block"])
    plt.title("RL Agent Actions Distribution")
    plt.xlabel("Action")
    plt.ylabel("Count")
    plt.show()
plot_rl_agent_decisions(actions)

print(f"RL Decision: {['Ignore', 'Flag', 'Block'][action]}")


# choose the synthetic_eth_transactions (1).csv file while uploading

In [ ]:

plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Autoencoder Training Loss')
plt.legend()
plt.show()


In [ ]:
sns.histplot(df['anomaly_score'], bins=30, kde=True)
plt.axvline(df['anomaly_score'].quantile(0.99), color='r', linestyle='dashed', label='99th Percentile Threshold')
plt.title('Anomaly Score Distribution')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled_data)

plt.figure(figsize=(10, 5))

# K-Means Visualization
plt.subplot(1, 2, 1)
sns.scatterplot(x=pca_data[:, 0], y=pca_data[:, 1], hue=df['kmeans_cluster'], palette='viridis')
plt.title("K-Means Clustering")

# DBSCAN Visualization
plt.subplot(1, 2, 2)
sns.scatterplot(x=pca_data[:, 0], y=pca_data[:, 1], hue=df['dbscan_cluster'], palette='coolwarm')
plt.title("DBSCAN Clustering")

plt.show()


In [ ]:
import shap

# Initialize SHAP Explainer
explainer = shap.TreeExplainer(iso_forest)
shap_values = explainer.shap_values(scaled_data)

# Compute Mean Absolute SHAP Value for Each Feature
shap_importance = np.abs(shap_values).mean(axis=0)

# Sort Feature Importances
sorted_indices = np.argsort(shap_importance)[::-1]

# Plot Feature Importance
plt.figure(figsize=(10, 5))
sns.barplot(x=[numeric_features.columns[i] for i in sorted_indices], y=shap_importance[sorted_indices])
plt.xticks(rotation=45)
plt.title("Feature Importance (SHAP) - Isolation Forest")
plt.xlabel("Features")
plt.ylabel("Mean SHAP Value")
plt.show()



In [ ]:
fraud_tx = df[df['fraud_flag'] == 1].head(50)  # Get top fraud cases

G_sub = nx.DiGraph()
for _, row in fraud_tx.iterrows():
    G_sub.add_edge(row['sender'], row['receiver'], weight=row['amount'])

plt.figure(figsize=(10, 6))
nx.draw(G_sub, with_labels=True, node_color='red', edge_color='gray', node_size=500)
plt.title("Fraudulent Transaction Subgraph")
plt.show()


In [ ]:
print(df.columns)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset (first 200 rows)

df = pd.read_csv('/content/synthetic_eth_transactions.csv', nrows=200)



# Function to compress Ethereum addresses (first and last character only)
def compress_address(address):
    return f"{address[0]}...{address[-1]}"

# Create a directed graph
G = nx.DiGraph()

# Add edges (transactions) to the graph
for _, row in df.iterrows():
    sender = compress_address(str(row['sender']))
    receiver = compress_address(str(row['receiver']))
    amount = row['amount']
    G.add_edge(sender, receiver, weight=amount)

# Draw the graph
plt.figure(figsize=(10, 6))
pos = nx.spring_layout(G, seed=42)  # Fixed layout for consistent visualization
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2000, edge_color='gray', width=2, font_size=10, font_weight='bold')

# Draw edge labels (transaction amounts)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# Show the graph
plt.title("Blockchain Transaction Graph (First 200 Rows)")
plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import joblib
joblib.dump(model, "fraud_model.joblib")


In [ ]:
import onnxmltools
onnx_model = onnxmltools.convert_sklearn(model)
onnxmltools.utils.save_model(onnx_model, "fraud_model.onnx")


In [ ]:
from fastapi import FastAPI
import joblib
import numpy as np

app = FastAPI()
model = joblib.load("fraud_model.joblib")

@app.post("/predict/")
def predict(data: list):
    prediction = model.predict(np.array(data).reshape(1, -1))
    return {"fraudulent": bool(prediction[0])}
